In [1]:
import nbimporter
import config
import requests
import json
import os
import pandas as pd

In [2]:
FSQ_API_KEY = os.environ['FOURSQUARE_API_KEY']
YELP_API_KEY = os.environ['YELP_API_KEY']
# GOOGLE_API_KEY = os.environ['']
NE = '45.693164,-73.489801'
SW = '45.433221,-73.898889'
CATEGORY = '13387'
AIM_AREA = NE, SW  # Downtown, MONTREAL
POI_RICH_DATA_FSQ = 'fsq_id,name,location,website,rating,stats,popularity'


In [5]:
def get_req(url, params, key):
    headers = {
        'Access': 'application/json',
        'Authorization': key,
    }
    response = requests.get(url=url, params=params, headers=headers)
    if response:
        return response.json()
    else:
        return response

In [27]:
params = {
    'ne': NE,
    'sw': SW,
    'categories': CATEGORY,
    'limit': '50',
    'sort': 'RATING' 
}
url = 'https://api.foursquare.com/v3/places/search?'

In [28]:
response = get_req(url, params, FSQ_API_KEY)

In [31]:
fsq_ids_fsq = [result['fsq_id'] for result in response['results']]
fsq_ids_fsq[0]

'4cec2a8af9916dcbf1f56a35'

### Loop through fsq_ids_fsqs => creates a list of json responses => pass to json_normalize and its beatiful
##### Question: should we be creating database first or pandas and then database

In [7]:
params = {
    'fields': POI_RICH_DATA_FSQ
}
# url = f"https://api.foursquare.com/v3/places/{fsq_ids_fsq[0]}"
url = f"https://api.foursquare.com/v3/places/4cec2a8af9916dcbf1f56a35"

In [8]:
response = get_req(url, params, FSQ_API_KEY)

In [14]:
r = [response, response, response]

In [11]:
# data = pd.json_normalize(r)

In [78]:
data

,fsq_id,name,popularity,rating,verified,website,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,stats.total_photos,stats.total_ratings,stats.total_tips
0,4cec2a8af9916dcbf1f56a35,Château Taillefer Lafon,0.969886,8.3,False,http://www.chateautailleferlafon.ca/,1500 Champagne Montee,CA,,"1500 Champagne Montee, Laval QC H7X 4H9",Laval,[L'Orée des Bois],H7X 4H9,QC,17,12,2
1,4cec2a8af9916dcbf1f56a35,Château Taillefer Lafon,0.969886,8.3,False,http://www.chateautailleferlafon.ca/,1500 Champagne Montee,CA,,"1500 Champagne Montee, Laval QC H7X 4H9",Laval,[L'Orée des Bois],H7X 4H9,QC,17,12,2
2,4cec2a8af9916dcbf1f56a35,Château Taillefer Lafon,0.969886,8.3,False,http://www.chateautailleferlafon.ca/,1500 Champagne Montee,CA,,"1500 Champagne Montee, Laval QC H7X 4H9",Laval,[L'Orée des Bois],H7X 4H9,QC,17,12,2


In [19]:
[result for result in r]

[{'fsq_id': '4cec2a8af9916dcbf1f56a35',
  'location': {'address': '1500 Champagne Montee',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '1500 Champagne Montee, Laval QC H7X 4H9',
   'locality': 'Laval',
   'neighborhood': ["L'Orée des Bois"],
   'postcode': 'H7X 4H9',
   'region': 'QC'},
  'name': 'Château Taillefer Lafon',
  'popularity': 0.969886055344547,
  'rating': 8.3,
  'stats': {'total_photos': 17, 'total_ratings': 12, 'total_tips': 2},
  'website': 'http://www.chateautailleferlafon.ca/'},
 {'fsq_id': '4cec2a8af9916dcbf1f56a35',
  'location': {'address': '1500 Champagne Montee',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '1500 Champagne Montee, Laval QC H7X 4H9',
   'locality': 'Laval',
   'neighborhood': ["L'Orée des Bois"],
   'postcode': 'H7X 4H9',
   'region': 'QC'},
  'name': 'Château Taillefer Lafon',
  'popularity': 0.969886055344547,
  'rating': 8.3,
  'stats': {'total_photos': 17, 'total_ratings': 12, 'total_tips': 2},
  'w

In [20]:
data_to_insert = [(str(result.get('fsq_id', None)), 
                      str(result.get('name', None)),
                       float(result.get('popularity', None)),
                        float(result.get('rating', None)),
                         str(result.get('website', None)),
                          str(result['location'].get('formatted_address', None)),
                           int(result['stats'].get('total_ratings', None)))
                               for result in r]
       

In [38]:
query = ''
for new_query in data_to_insert:
    if new_query == data_to_insert[-1]:
        query += str(new_query)
    else:
        query += str(new_query) + ','

In [40]:
print(f"""
INSERT INTO
  users (fsq_id, name, popularity, rating, website, formatted_address, total_ratings)
VALUES
    {query};

""")




INSERT INTO
  users (fsq_id, name, popularity, rating, website, formatted_address, total_ratings)
VALUES
    ('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12)('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12)('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12);




"('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12)('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12)('4cec2a8af9916dcbf1f56a35', 'Château Taillefer Lafon', 0.969886055344547, 8.3, 'http://www.chateautailleferlafon.ca/', '1500 Champagne Montee, Laval QC H7X 4H9', 12)"

In [26]:
def yelp_it(url, params=None):
    headers = {
    'Access': 'application/json',
    'Authorization': f'Bearer {YELP_API_KEY}',
    }
    params = params or {}
    response = requests.get(url=url, params=params, headers=headers)
    if response:
        return response.json()
    else:
        return response

In [24]:
url = 'https://api.yelp.com/v3/businesses/search'
params = {
    'latitude': '45.502772',
    'longitude': '-73.552406',
    'limit': 50,
    'radius': 1000,
    'categories': 'cocktailbars',
    'sort_by': 'rating',
    
    }
response = yelp_it(url, params=params)

In [20]:
response.url

'https://api.yelp.com/v3/businesses/search?latitude=45.502772&longitude=-73.552406&limit=50&radius=1000&categories=cocktailbars&sort_by=rating'

In [22]:
r = response.json()['businesses']
ids_yelp = [result['id'] for result in r]

In [33]:
ids_yelp = [result['id'] for result in r]

In [47]:
url = f'https://api.yelp.com/v3/businesses/BCSofuQwu1VE8wjzDaQ3qQ'

res = yelp_it(url)

In [56]:
res = (res, res, res)
# (fsq_id, name, popularity, rating, website, formatted_address, total_ratings)

In [57]:
type(res)

tuple

In [58]:
data_to_insert = [(result.get('id', None), 
                      result.get('name', None),
                       result.get('rating', None),
                        result.get('review_count', None),
                         result.get('url', None),
                          result.get('location', {}).get('display_address', None),
                           result.get('price', None))
                               for result in res]

In [59]:
data_to_insert[0]

('BCSofuQwu1VE8wjzDaQ3qQ',
 'The Coldroom',
 4.5,
 207,
 'https://www.yelp.com/biz/the-coldroom-montr%C3%A9al-2?adjust_creative=9i_-1YvTzQ430GyCdr-vpw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=9i_-1YvTzQ430GyCdr-vpw',
 ['Rue Saint-Vincent', 'Montreal, QC H2Y 1G8', 'Canada'],
 '$$')

In [60]:
data_to_insert = [(result.get('id', None), 
                      result.get('name', None),
                       result.get('rating', None),
                        result.get('review_count', None),
                         result.get('url', None),
                          result.get('location', {}).get('display_address', None),
                           result.get('price', None))
                               for result in r]

In [62]:
data_to_insert[0]

('BCSofuQwu1VE8wjzDaQ3qQ',
 'The Coldroom',
 4.5,
 207,
 'https://www.yelp.com/biz/the-coldroom-montr%C3%A9al-2?adjust_creative=9i_-1YvTzQ430GyCdr-vpw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9i_-1YvTzQ430GyCdr-vpw',
 ['Rue Saint-Vincent', 'Montreal, QC H2Y 1G8', 'Canada'],
 '$$')